# Funcionalidades

In [1]:
from ultralytics import YOLO

In [19]:
# Cargar modelo YOLO11s fine-tuned
model = YOLO('C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\modelos\\best.pt')

##  Cantidad de clientes por zona

In [72]:
# Funcion para determinar coordenadas de objetos en videos
import cv2

# Función de callback para obtener coordenadas al hacer clic
def obtener_coordenadas(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Solo muestra al hacer clic izquierdo
        print(f'Coordenadas: ({x}, {y})')

# Capturar un frame del video
cap = cv2.VideoCapture('C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\grabaciones\\grabacion_2025-02-21_19-42-20.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    cv2.imshow('Frame', frame)
    cv2.setMouseCallback('Frame', obtener_coordenadas)  # Asigna la función al frame

    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No se pudo leer el frame del video.")



Coordenadas: (199, 115)
Coordenadas: (424, 285)


In [12]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

In [94]:
# Función para calcular la distancia entre un punto (px, py) y una línea definida por dos puntos (x1, y1) y (x2, y2)
def distancia_punto_linea(px, py, x1, y1, x2, y2):
    return abs((y2 - y1) * px - (x2 - x1) * py + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

In [93]:
# Definir las coordenadas de la línea del mostrador (inicio y fin)
x1_mostrador, y1_mostrador = 199, 115
x2_mostrador, y2_mostrador = 424, 285

In [ ]:
# Función para contar los clientes cerca de la línea del mostrador
def contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador):
    count = 0
    clientes_detectados = []  # Lista para almacenar las coordenadas de los clientes detectados
    for deteccion in detecciones:
        bbox = deteccion.xywh[0]
        x, y = bbox[0].item(), bbox[1].item()  # Centro en formato xywh
        clase = int(deteccion.cls[0].item())

        if clase == 1:
            distancia = distancia_punto_linea(x, y, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)
            if distancia < 30:  # Umbral para considerar que está en la zona del mostrador
                count += 1
                clientes_detectados.append((x, y))  # Guardar la coordenada de los clientes

    return count, clientes_detectados

In [103]:
# Leer video
import cv2
video = cv2.VideoCapture("C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\grabaciones\\grabacion_2025-02-21_19-42-20.mp4")

# Lista para almacenar los resultados
resultados = []

# Extraer la fecha y hora inicial desde el nombre del archivo
nombre_video = "grabacion_2025-02-21_19-42-20.mp4"
fecha_hora_inicial_str = nombre_video.split('_')[1] + " " + nombre_video.split('_')[2].replace('.mp4', '')
fecha_hora_inicial = datetime.strptime(fecha_hora_inicial_str, "%Y-%m-%d %H-%M-%S")

In [ ]:

# crea un csv con la cantidad de clientes cerca de mostrador detectados

# Procesar cada fotograma
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Obtener el número de fotograma y calcular el tiempo en segundos
    frame_number = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    fps = video.get(cv2.CAP_PROP_FPS)
    segundos_transcurridos = frame_number / fps

    # Calcular la nueva fecha y hora
    nueva_fecha_hora = fecha_hora_inicial + timedelta(seconds=segundos_transcurridos)
    fecha_hora_exacta = nueva_fecha_hora.strftime("%Y-%m-%d %H:%M:%S")

    # Realizar la detección de objetos (clientes y empleados)
    resultados_deteccion = model(frame)

    # Extraer las detecciones
    detecciones = resultados_deteccion[0].boxes if resultados_deteccion else []

    # Contar clientes cerca del mostrador y obtener sus coordenadas
    clientes_mostrador, clientes_detectados = contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)

    # Guardar la información del primer cliente detectado (si hay alguno)
    if clientes_detectados:
        x_cliente, y_cliente = clientes_detectados[0]  # Tomar la primera detección

        # Guardar la detección en la lista
        resultados.append([fecha_hora_exacta, clientes_mostrador, x_cliente, y_cliente])

# Guardar los resultados en un CSV
df = pd.DataFrame(resultados, columns=['Fecha Hora y Segundo', 'Clientes Cerca del Mostrador', 'X Cliente', 'Y Cliente'])
df.to_csv('resultados_deteccion_mostrador_con_ubicaciones.csv', index=False)

# Liberar el video y cerrar ventanas
video.release()
cv2.destroyAllWindows()

0: 384x640 4 clientes, 2 mostradors, 350.2ms
Speed: 8.9ms preprocess, 350.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 303.3ms
Speed: 2.1ms preprocess, 303.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 325.0ms
Speed: 2.2ms preprocess, 325.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 323.9ms
Speed: 2.6ms preprocess, 323.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 283.2ms
Speed: 3.4ms preprocess, 283.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 343.0ms
Speed: 2.4ms preprocess, 343.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 477.1ms
Speed: 2.6ms preprocess, 477.1ms inference, 1.9ms postprocess per image a

In [ ]:

# crea un csv con la cantidad de clientes detectados en total y los que estan cerca de mostrador
resultados_clientes_totales = []

# Procesar cada fotograma
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Obtener el número de fotograma y calcular el tiempo en segundos
    frame_number = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    fps = video.get(cv2.CAP_PROP_FPS)
    segundos_transcurridos = frame_number / fps

    # Calcular la nueva fecha y hora
    nueva_fecha_hora = fecha_hora_inicial + timedelta(seconds=segundos_transcurridos)
    fecha_hora_exacta = nueva_fecha_hora.strftime("%Y-%m-%d %H:%M:%S")

    # Realizar la detección de objetos (clientes y empleados)
    resultados_deteccion = model(frame)

    # Extraer las detecciones
    detecciones = resultados_deteccion[0].boxes if resultados_deteccion else []

    # Contar clientes cerca del mostrador y obtener sus coordenadas
    clientes_mostrador, clientes_detectados = contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)

    # Contar todos los clientes detectados en el fotograma
    clientes_totales = sum(1 for deteccion in detecciones if int(deteccion.cls[0].item()) == 1)  # Suponiendo que 1 es la clase "cliente"

    # Guardar la información del primer cliente detectado (si hay alguno)
    if clientes_detectados:
        x_cliente, y_cliente = clientes_detectados[0]  # Tomar la primera detección

        # Guardar la detección en la lista de resultados
        resultados_clientes_totales.append([fecha_hora_exacta, clientes_mostrador, clientes_totales, x_cliente, y_cliente])

# Guardar los resultados de detección con clientes totales en un CSV
df_clientes_totales = pd.DataFrame(resultados_clientes_totales, columns=['Fecha Hora y Segundo', 'Clientes Cerca del Mostrador', 'Clientes Totales', 'X Cliente', 'Y Cliente'])
df_clientes_totales.to_csv('resultados_clientes_totales.csv', index=False)

# Liberar el video y cerrar ventanas
video.release()
cv2.destroyAllWindows()


0: 384x640 4 clientes, 2 mostradors, 351.3ms
Speed: 13.4ms preprocess, 351.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 311.6ms
Speed: 2.6ms preprocess, 311.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 377.6ms
Speed: 3.4ms preprocess, 377.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 390.8ms
Speed: 2.5ms preprocess, 390.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 373.2ms
Speed: 2.6ms preprocess, 373.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 444.7ms
Speed: 2.3ms preprocess, 444.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 307.7ms
Speed: 2.4ms preprocess, 307.7ms inference, 1.5ms postprocess per image

In [118]:
import cv2

# Cargar el video
video = cv2.VideoCapture("C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\grabaciones\\grabacion_2025-02-21_19-42-20.mp4")

# Saltar los primeros 4 frames para llegar al quinto
frame_count = 0
while frame_count < 100:
    ret, frame = video.read()
    frame_count += 1
    if not ret:
        print("No se pudo leer el frame 5.")
        video.release()
        exit()

# Definir la coordenada del objeto detectado (según el CSV)
x, y = 436.643310546875,313.2765808105469 # Centro de la caja

# Dibujar el punto del objeto detectado
cv2.circle(frame, (int(x), int(y)), 5, (255, 0, 0), -1)  # Azul

# Dibujar la línea del mostrador
cv2.line(frame, (x1_mostrador, y1_mostrador), (x2_mostrador, y2_mostrador), (0, 0, 255), 2)  # Rojo

# Mostrar el frame
cv2.imshow("Detección", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Liberar el video
video.release()
